In [3]:
#导入各种需要的库
#导入各种需要的库
import geemap
import ee
Map = geemap.Map()
from IPython.display import Image
import PySimpleGUI as sg
from tqdm.notebook import tqdm
from time import sleep
import csv
import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [*] %(processName)s %(message)s"
)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:

Beijing = ee.Geometry.Polygon(
          [[[116.29730339279362, 39.99599891062294],
            [116.29730339279362, 39.84222494267003],
            [116.48681755294987, 39.84222494267003],
            [116.48681755294987, 39.99599891062294]]], None, False)
roi = Beijing
Map.addLayer(roi, {'color':'grey'}, 'studyArea')
Map.centerObject(roi)

year = 2020

# reomove cloud for Landsat-8
def rmL8Cloud(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                 .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask).toFloat().divide(10000) \
              .copyProperties(image) \
              .copyProperties(image, ['system:time_start','system:time_end','system:footprint'])

l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
           .filterBounds(roi) \
           .filterDate(ee.Date.fromYMD(year,1,1),ee.Date.fromYMD(year+1,1,1)) \
           .map(rmL8Cloud) \
           .select(['B2','B3','B4','B5','B6'])

# year composite
l8_year =  l8.median().clip(roi)
print("l8_year",l8_year)

# half-year composite

def func_wxg(n):
    start = ee.Date.fromYMD(year,1,1).advance(n, 'month')
    end = start.advance(6, 'month')
    tmpMedian = l8.filterDate(start, end).median().clip(roi).set("system:time_start", start.millis())
    return tmpMedian

l8_halfyear = ee.List.sequence(0, 1*11,6).map(func_wxg).flatten()
l8_halfyear = ee.ImageCollection.fromImages(l8_halfyear)
print("l8_halfyear",l8_halfyear)

# season composite

def func_coc(n):
    start = ee.Date.fromYMD(year,1,1).advance(n, 'month')
    end = start.advance(3, 'month')
    tmpMedian = l8.filterDate(start, end).median().clip(roi).set("system:time_start", start.millis())
    return tmpMedian

l8_season = ee.List.sequence(0, 1*11,3).map(func_coc).flatten()

l8_season = ee.ImageCollection.fromImages(l8_season)
print("l8_season",l8_season)

# month composite

def func_oew(n):
    start = ee.Date.fromYMD(year,1,1).advance(n, 'month')
    end = start.advance(1, 'month')
    tmpMedian = l8.filterDate(start, end).median().clip(roi).set("system:time_start", start.millis())
    return tmpMedian

l8_month = ee.List.sequence(0, 1*11,1).map(func_oew).flatten()


l8_month = ee.ImageCollection.fromImages(l8_month)
print("l8_month",l8_month)

# l8_halfmonth

def func_wde(item):
    start_date = ee.Date.fromYMD(year, item, 1)
    mid_date = start_date.advance(15,'day')
    end_date = start_date.advance(1,'month')
    pre_month_img = l8.filterDate(start_date,mid_date).mean() \
                            .set('system:time_start',start_date.millis()) \
                            .set('time',start_date.format('YYYY-MM-dd'))
    pro_month_img = l8.filterDate(mid_date,end_date).mean() \
                            .set('system:time_start',start_date.millis()) \
                            .set('time',mid_date.format('YYYY-MM-dd'))
return ee.ImageCollection([pre_month_img,pro_month_img])

l8_halfmonth = ee.List.sequence(1,12).map(func_wde)

# define iterate function
def accumulate(imageCol, list):
    return ee.ImageCollection(list).merge(imageCol)

# flatten each element in the l8_halfmonth collection
l8_halfmonth = l8_halfmonth.iterate(accumulate,ee.ImageCollection([]))
l8_halfmonth= ee.ImageCollection(l8_halfmonth)
print("l8_halfmonth",l8_halfmonth)

# show the image
visParam = {'bands':['B6', 'B5', 'B4'], 'min':0, 'max':0.3}
Map.addLayer(l8_year, visParam, 'l8_year')
MapMap
